# Importing the Libraries

In [ ]:
import pandas as pd
from sklearn import linear_model
from sklearn import preprocessing

# Importing the Training Data

In [ ]:
df = pd.read_csv("titanic_train.csv")       # read the data
df

# Dropping Irrelevant Fields

In [ ]:
# drop the columns that are outright not useful
df.drop(labels = ['PassengerId',
         'Name',
         'Ticket',
         'Fare',
         'Cabin'], 
        axis = 1, 
        inplace = True)

# check to see if the colummns are removed
df

In [ ]:
df.isna().sum()

# Removing Rows with Empty Cells

In [ ]:
df.dropna(inplace=True)                   # drop all rows with NaN
df.reset_index(inplace=True, drop=True)   # re-index the dataframe
df


# Encoding the Features 

In [ ]:
# initialize label encoder
label_encoder = preprocessing.LabelEncoder()

# convert Sex and Embarked features to numeric
df["Sex"] = label_encoder.fit_transform(df["Sex"])
# 0 = female
# 1 = male

df["Embarked"] = label_encoder.fit_transform(df["Embarked"])
# 0 = C
# 1 = Q
# 2 = S

df

# Feature Engineering

In [ ]:
df['Alone'] = [1 if size==0 else 0 for size in (df['Parch'] + df['SibSp'])]
df

# Viewing Correlations

In [ ]:
df.corr(method='spearman')

In [ ]:
df.corr(method='pearson')

In [ ]:
df.corr(method='pearson')['Survived'].abs().sort_values(ascending=False)


# Dropping Columns (again)

In [ ]:
# drop the columns that are not useful to us
df.drop(columns=['SibSp','Parch'], inplace=True)

# check to see if the colummns are removed
df

# Examining VIF

In [ ]:
from sklearn.linear_model import LinearRegression
    
def sklearn_vif(exogs, data):    
    # initialize dictionaries
    vif_dict, tolerance_dict = {}, {}

    # form input data for each exogenous variable
    for exog in exogs:
        not_exog = [i for i in exogs if i != exog]
        X, y = data[not_exog], data[exog]

        # extract r-squared from the fit
        r_squared = LinearRegression().fit(X, y).score(X, y)

        # calculate VIF
        vif = 1/(1 - r_squared)
        vif_dict[exog] = vif

        # calculate tolerance
        tolerance = 1 - r_squared
        tolerance_dict[exog] = tolerance

    # return VIF DataFrame
    return pd.DataFrame({'VIF': vif_dict, 'Tolerance': tolerance_dict})

In [ ]:
sklearn_vif(data=df, 
            exogs=['Pclass','Sex','Age','Embarked','Alone'])

# Making Fields Categorical

In [ ]:
# make fields categorical
df["Sex"]      = pd.Categorical(df["Sex"])
df["Embarked"] = pd.Categorical(df["Embarked"])
df["Survived"] = pd.Categorical(df["Survived"])
df["Pclass"]   = pd.Categorical(df["Pclass"])
df["Alone"]    = pd.Categorical(df["Alone"])

print(df.dtypes)     # examine the datatypes for each feature

In [ ]:
df

# Saving a Copy of the Modified Dataframe

In [ ]:
# save a copy for use later
df.to_pickle('df_titanic.pickle')

In [ ]:
df.to_csv("cleaned_titanic.csv")

In [ ]:
df

# Splitting the DataSet into Training and Testing Sets

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set, train_labels, test_labels = train_test_split(
                              df.iloc[:,1:],           # features (all except Survived)
                              df.iloc[:,0],            # label (Survived)     
                              test_size = 0.25,        # split ratio
                              random_state = 1,        # set random seed
                              stratify = df.iloc[:,0]) # randomize based on labels
print(train_set.shape)
print(train_labels.shape)

In [ ]:
test_set.shape
test_labels.shape

In [ ]:
train_set

In [ ]:
train_labels

In [ ]:
print(test_set.shape)
print(test_labels.shape)


In [ ]:
test_set

In [ ]:
test_labels

# Training the Model

In [ ]:
# initialize logistic regression model
log_regress = linear_model.LogisticRegression()

# train the model
log_regress.fit(X = train_set,
                y = train_labels)


# Getting the Intercept and Coefficient

In [ ]:
# print trained model intercept
print(log_regress.intercept_)

# print trained model coefficients
print(log_regress.coef_)


# Making Predictions

In [ ]:
# get the probability estimates
preds_prob = pd.DataFrame(log_regress.predict_proba(X=test_set))
# assign column names to prediction
preds_prob.columns = ["DeathProb", "SurvivalProb"]

# get the predicted class labels
preds_class = pd.DataFrame(log_regress.predict(X=test_set))
preds_class.columns = ["Survived?"]

# merge the two dataframes into one
result = pd.concat([preds_prob, preds_class], axis=1)
result


# Getting the Accuracy of the Predictions

In [ ]:
# make predictions
preds = log_regress.predict(X=test_set)

# generate table of predictions vs actual
pd.crosstab(preds, test_labels)

In [ ]:
# get the accuracy of the prediction
log_regress.score(X = test_set ,
                  y = test_labels)


In [ ]:
(90+52) / (90+52+16+20)

# Viewing the Confusion Matrix

In [ ]:
from sklearn import metrics

# view the confusion matrix
metrics.confusion_matrix(y_true = test_labels,  # True labels
                         y_pred = preds)        # Predicted labels


# Viewing Summary of Classification Metrics

In [ ]:
# View summary of common classification metrics
print(metrics.classification_report(
      y_true = test_labels,
      y_pred = preds))

# Using a Testing Dataset for Prediction

In [ ]:
# read the data for testing the model
df_test = pd.read_csv("titanic_test.csv")

df_test['Alone'] = [1 if size == 0 else 0 for size in (df_test['Parch'] + df_test['SibSp'])]

# drop the columns that are not useful to us
df_test.drop(['PassengerId',
              'Name',
              'Ticket',
              'Fare',
              'Cabin',
              'SibSp',
              'Parch'], axis=1, inplace=True)

df_test.dropna(inplace=True)            # drop all rows with NaN
df_test.reset_index(inplace=True, drop=True)

# label encode Sex and Embarked
df_test["Sex"]      = label_encoder.fit_transform(df_test["Sex"])
df_test["Embarked"] = label_encoder.fit_transform(df_test["Embarked"])

# make fields categorical
df_test["Sex"]      = pd.Categorical(df_test["Sex"])
df_test["Embarked"] = pd.Categorical(df_test["Embarked"])
df_test["Pclass"]   = pd.Categorical(df_test["Pclass"])
df_test["Alone"]   = pd.Categorical(df_test["Alone"])

# make test set predictions
test_preds = log_regress.predict(X=df_test)

# get prediction probabilities
test_preds_prob = log_regress.predict_proba(X=df_test)

# convert into dataframe
df_prob = pd.DataFrame(test_preds_prob, columns=['Death', 'Survived'])

# add predictions, death and survival probabilities to the dataframe
df_test["Prediction"] = test_preds
df_test["Death"]      = df_prob["Death"]
df_test["Survived"]   = df_prob["Survived"]

df_test


# Making a Single Prediction

In [ ]:
# Define feature names
feature_names = ['Pclass', 'Sex', 'Age', 'Embarked', 'Alone']

# Create a DataFrame with the feature values
test_data = pd.DataFrame([[1,0,20,1,1]], columns=feature_names)

# Pclass = 1, Sex = Female, Age = 20, Embarked = Q, Alone = 1 (yes)
print(log_regress.predict_proba(test_data)) # predict a probability
print(log_regress.predict(test_data))       # predict a label


In [ ]:
df